In [1]:
import os
import torch
import Self_net_architecture
import tifffile as tif
import numpy as np

In [2]:
def tensor_info(adrr) -> None:
    print(f'shape: {adrr.shape} \ndtype: {adrr.dtype} \nmax: {adrr.max()} \nmin: {adrr.min()} \nmean: {adrr.mean()} \nstd: {adrr.std()}')

# Model

In [3]:
input_nc = 1
output_nc = 1
device=torch.device('cuda:0')

In [4]:
deblur_net=Self_net_architecture.define_G(input_nc=input_nc, output_nc=output_nc, ngf=32, netG='mwcnn', device=device,use_dropout=False,norm='instance')
deblur_net.load_state_dict(torch.load('/home/ryuuyou/Project/self_net/data/visor/checkpoint/saved_models/deblur_net/50_250.pkl'))

initialize network with normal


<All keys matched successfully>

# Data

In [5]:
data_path = '/home/ryuuyou/Project/self_net/data/visor/raw_data/test'
data_name_list = os.listdir(data_path)
data_name_list.sort(key=lambda name:int(name.split('.')[0]))

result_save_path = '/home/ryuuyou/Project/self_net/data/visor/reconstruction'
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [6]:
print(data_name_list)

['1000.tif', '1001.tif', '1002.tif', '1003.tif', '1004.tif', '1005.tif', '1006.tif', '1007.tif', '1008.tif', '1009.tif', '1010.tif', '1011.tif', '1012.tif', '1013.tif', '1014.tif', '1015.tif', '1016.tif']


In [7]:
deblur_net.eval()
res_stack=[]
for index, name in enumerate(data_name_list):
    img = tif.imread(os.path.join(data_path, name))
    img = np.expand_dims(np.expand_dims(img, axis=0), axis=0)
    img = img.astype(np.float32)
    img = (img - img.min())/(img.max() - img.min())
    img_tensor = torch.from_numpy(img).to(device=device).to(torch.float32)
    with torch.no_grad():
        res = deblur_net(img_tensor)
    res_npadrr = res.squeeze_(0).squeeze_(0).cpu().numpy()
    tif.imwrite(os.path.join(result_save_path, name), res_npadrr)